<a href="https://colab.research.google.com/github/Ptkiuo/file-upload/blob/main/PatchVer_JP_ByMenow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on 4chan NovelAILeaks (naifu)[src](https://boards.4channel.org/g/thread/89095460#p89097704)

updated on 07/11/2022 16.49 GMT +7


In [ ]:
#@title ### 0. GPUが使用できるかをチェックします
!nvidia-smi

In [ ]:
#@title ### 1. Novel AIのAPIをダウンロードします。
#@markdown もし、ダウンロードが遅かったらリトライしてください。

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

In [3]:
#@title ### 2. 依存関係（モジュール）をインストールします
#@markdown インストールが長いですが、気長に待ってください

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared
!/content/naifu/venv/bin/python -m pip install -qq pytorch_lightning==1.7.7

/content/naifu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 32.1 MB/s 
     |████████████████████████████████| 468 kB 67.2 MB/s 
+ virtualenv venv
created virtual environment CPython3.7.15.final.0-64 in 932ms
  creator CPython3Posix(dest=/content/naifu/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.3, setuptools==65.5.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
+ . venv/bin/activate
++ '[' venv/bin/activate = ./setup.sh ']'
++ deactivate nondestructive
++ unset -f pydoc
++ '[' -z '' ']'
++ '[' -z '' ']'
++ hash -r
++ '[' -z '' ']'
++ unset VIRTUAL_ENV
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/naifu/venv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 33.0M  100 33.0M    0     0  48.9M      0 --:--:-- --:--:-- --:--:-- 48.9M
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 33.1 MB/s eta 0:00:00


In [ ]:
#@title ### 3. サイトをホストする準備をし、スタートします！

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

/content/naifu
2022-11-12T03:30:02Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2022-11-12T03:30:02Z INF Requesting new quick Tunnel on trycloudflare.com...
2022-11-12T03:30:02.934325Z  INFO bore_cli::client: connected to server remote_port=34431
2022-11-12T03:30:02.934382Z  INFO bore_cli::client: listening at bore.pub:34431
Starting Hydra Node HTTP TOKEN=None
2022-11-12 03:30:04,395 INFO config.py(603) - CPU: Intel(R) Xeon(R) CPU @ 2.00GHz
2022-11-12 03:30:04,395 INFO config.py(603) - GPU: Tesla T4
2022-11-12 03:30:04,396 INFO config.py(603) - GPU RAM: 15gb
2022-11-12 03:30:04,396 INFO config.py(603) - MODEL: stable-diffusion
2022-11-12 03:30:04,396 INFO mo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ### 4. (必須ではありません)

%cd /content/
!mkdir /content/naifu/models/animefull-latest
#!tar xf animefulfull-latest && rm animefull-lal-latest.tar -C /content/naifu/models/animetest.tar
#!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!tar xf animefull-latest.tar -C /content/naifu/models/animefull-latest && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969